# This notebook explores the census tract food access data and compares it to community area adult obesity rates. #

In [1]:
import numpy as np
import pandas as pd

import cleantools as ct

In [2]:
food_data = pd.read_csv('./data/food_data.csv')
census_to_community = pd.read_csv('./data/chicago_census_tract_to_community_area.csv')
obesity_rate_df = pd.read_csv('./data/chicago_health_atlas_adult_obesity_rate.csv')
diabetes_rate_df = pd.read_csv('./data/chicago_health_diabetes_rate.csv')

In [3]:
food_data.shape

(1314, 160)

In [4]:
census_to_community.shape

(794, 3)

In [5]:
obesity_rate_df.shape

(75, 5)

In [6]:
diabetes_rate_df.shape

(61, 5)

Food data is provided on the census tract level, and only tracts in Cook County are included in the food_data DataFrame.  The census_to_community DataFrame identifies the Community Area that each census tract in Chicago is a part of.  Note that it does not include all census tracts in food_data, since Cook County is larger than Chicago.

Obesity rate data is missing for three community areas, and diabetes rate data is missing for 17 community areas.

### Section A: Cleaning Health Data ###

In [7]:
diabetes_rate_df.isna().sum()

Layer                    1
Name                     1
GEOID                    1
HCSDIAP_2016-2018        0
HCSDIAP_2016-2018_moe    0
dtype: int64

In [8]:
obesity_rate_df.isna().sum()

Layer                   1
Name                    1
GEOID                   1
HCSOBP_2016-2018        0
HCSOBP_2016-2018_moe    0
dtype: int64

In [9]:
diabetes_rate_df.head()

,Layer,Name,GEOID,HCSDIAP_2016-2018,HCSDIAP_2016-2018_moe
0,NaN,NaN,NaN,"Adult diabetes rate (% of adults), 2016-2018",(90% margin of error)
1,Community area,Rogers Park,1.0,11.2,4.741964285345
2,Community area,Norwood Park,10.0,9.3,4.8678571420850005
3,Community area,Jefferson Park,11.0,9.8,7.30178571395
4,Community area,North Park,13.0,17.3,10.994642856505001


The reason the data has nulls is because some of the header took two rows. Let's drop that extra row.

In [10]:
diabetes_rate_df.drop(0, inplace = True)

In [11]:
diabetes_rate_df.dtypes

Layer                     object
Name                      object
GEOID                    float64
HCSDIAP_2016-2018         object
HCSDIAP_2016-2018_moe     object
dtype: object

In [12]:
obesity_rate_df.drop(0, inplace=True)

In [13]:
obesity_rate_df.dtypes

Layer                    object
Name                     object
GEOID                   float64
HCSOBP_2016-2018         object
HCSOBP_2016-2018_moe     object
dtype: object

In [14]:
# Convert GEOID to int.  This will be necessary later when merging data.
def float_to_int(num):
    return int(num)

diabetes_rate_df['GEOID'] = diabetes_rate_df['GEOID'].apply(float_to_int)
obesity_rate_df['GEOID'] = obesity_rate_df['GEOID'].apply(float_to_int)

In [15]:
# Convert the rows that should be numeric but were not coded as such because of the extra header to float.
diabetes_rate_df['HCSDIAP_2016-2018'] = diabetes_rate_df['HCSDIAP_2016-2018'].apply(ct.convert_to_float)
diabetes_rate_df['HCSDIAP_2016-2018_moe'] = diabetes_rate_df['HCSDIAP_2016-2018_moe'].apply(ct.convert_to_float)
obesity_rate_df['HCSOBP_2016-2018'] = obesity_rate_df['HCSOBP_2016-2018'].apply(ct.convert_to_float)
obesity_rate_df['HCSOBP_2016-2018_moe'] = obesity_rate_df['HCSOBP_2016-2018_moe'].apply(ct.convert_to_float)

### Section B: Merging Health Data with census_to_community ###

This process allows us to map health data for each community area to the individual census tracts within the community.

In [16]:
diabetes_rate_df['GEOID'].max()

77

In [17]:
obesity_rate_df['GEOID'].max()

77

In [18]:
census_to_community['Community Area'].max()

77

GEOID in the health data corresponds with community area in the census to community conversion table. Both contain values up to 77. However, the health data does not include every census tract that is in food_data.csv.

Before merging, we confirm no nulls.

In [19]:
census_to_community.isna().sum()

Tract             0
Label             0
Community Area    0
dtype: int64

In [20]:
obesity_rate_df.isna().sum()

Layer                   0
Name                    0
GEOID                   0
HCSOBP_2016-2018        0
HCSOBP_2016-2018_moe    0
dtype: int64

In [21]:
diabetes_rate_df.isna().sum()

Layer                    0
Name                     0
GEOID                    0
HCSDIAP_2016-2018        0
HCSDIAP_2016-2018_moe    0
dtype: int64

In [22]:
diabetes_rate_df.rename({'GEOID': 'Community Area'}, axis=1, inplace=True)
obesity_rate_df.rename({'GEOID': 'Community Area'}, axis=1, inplace=True)
diabetes_rate_by_census_df = pd.merge(left=census_to_community, right=diabetes_rate_df, how='left', on='Community Area')
obesity_rate_by_census_df = pd.merge(left=census_to_community, right=obesity_rate_df, how='left', on='Community Area')

In [23]:
diabetes_rate_by_census_df.isna().sum()

Tract                     0
Label                     0
Community Area            0
Layer                    63
Name                     63
HCSDIAP_2016-2018        63
HCSDIAP_2016-2018_moe    63
dtype: int64

In [24]:
obesity_rate_by_census_df.isna().sum()

Tract                   0
Label                   0
Community Area          0
Layer                   8
Name                    8
HCSOBP_2016-2018        8
HCSOBP_2016-2018_moe    8
dtype: int64

The missing health data results in null values for some census tracts.  These tracts need to be dropped from the respective data frames.

In [25]:
diabetes_rate_by_census_df[diabetes_rate_by_census_df['HCSDIAP_2016-2018'].isna()].head()

,Tract,Label,Community Area,Layer,Name,HCSDIAP_2016-2018,HCSDIAP_2016-2018_moe
143,17031090100,"Census Tract 901, Cook County, Illinois",9,NaN,NaN,NaN,NaN
144,17031090200,"Census Tract 902, Cook County, Illinois",9,NaN,NaN,NaN,NaN
145,17031090300,"Census Tract 903, Cook County, Illinois",9,NaN,NaN,NaN,NaN
159,17031120100,"Census Tract 1201, Cook County, Illinois",12,NaN,NaN,NaN,NaN
160,17031120200,"Census Tract 1202, Cook County, Illinois",12,NaN,NaN,NaN,NaN


In [26]:
diabetes_rate_by_census_df.dropna(inplace=True)
obesity_rate_by_census_df.dropna(inplace=True)

In [27]:
obesity_rate_by_census_df.columns

Index(['Tract', 'Label', 'Community Area', 'Layer', 'Name', 'HCSOBP_2016-2018',
       'HCSOBP_2016-2018_moe'],
      dtype='object')

In [28]:
diabetes_rate_by_census_df.columns

Index(['Tract', 'Label', 'Community Area', 'Layer', 'Name',
       'HCSDIAP_2016-2018', 'HCSDIAP_2016-2018_moe'],
      dtype='object')

Label, Layer, and Name provide basic information not useful for modelling.  Although margin of error is useful information, it is not appropriate for modelling.

In [29]:
obesity_rate_by_census_df.drop(columns=['Label', 'Layer', 'Name', 'HCSOBP_2016-2018_moe'], inplace=True)
diabetes_rate_by_census_df.drop(columns=['Label', 'Layer', 'Name', 'HCSDIAP_2016-2018_moe'], inplace=True)

### Section C: Cleaning Food Access Data ###

In [30]:
# Custom function creates a dataframe with information about the NULLs and datatypes of each column.
food_info = ct.info_frame(food_data)

In [31]:
food_info.sort_values(by='num_nans', ascending=False).head()

,name,type,num_nans
133,lahunv20share,float64,1314
94,lablack10,float64,1314
106,lahunv10,float64,1314
105,lahisp10share,float64,1314
103,laomultir10share,float64,1314


In [32]:
len(food_info[food_info['num_nans'] == 1314])

52

In [33]:
len(food_info[food_info['num_nans'] == 0])

49

This data has quite a few NULL values.  These values are actually coded as NULL in the original csv, suggeting that they are missing information or irrelevant for the selected census tract.  52 of the 160 columns are entirely NULL.  A total of 49 columns in this table have no NULL values at all.

In [34]:
null_cols = [food_info.loc[col]['name'] for col in food_info.index if food_info.loc[col]['num_nans'] == 1314]

In [35]:
# Drop the columns that have only NULL values, as well as Unnamed: 0
food_data.drop(columns=null_cols, inplace=True)
food_data.drop(columns='Unnamed: 0', inplace=True)

In [36]:
food_data.shape

(1314, 107)

In [37]:
food_info = ct.info_frame(food_data)

In [38]:
food_info.sort_values(by='num_nans', ascending=False).head()

,name,type,num_nans
30,LALOWI1_20,float64,974
28,LALOWI1_10,float64,974
27,LAPOP1_20,float64,974
25,LAPOP1_10,float64,974
81,lasnap1,float64,972


In [39]:
food_data['LAPOP05_10'].value_counts()

0.0       48
3.0        4
1.0        4
4.0        4
2186.0     3
          ..
3149.0     1
2424.0     1
57.0       1
2985.0     1
1101.0     1
Name: LAPOP05_10, Length: 799, dtype: int64

At least some columns with NULL values also have some 0 values, so it's reasonable to assume that NULL does not mean 0.  It's also reasonable to assume that it doesn't indicate the feature is somehow irrelevant for a given census tract, as these correspond to values such as the portion of low-access, low-income population at certain distances from a supermarket.  Since no further information is provided about these NULL values, it seems most likely that the data was not available.

Since NULL values will be problematic for any method of food desert identification, and we want to include every census tract possible in our analysis, let's examine the features that have no NULL values at all.  These features alone may provide enough information for our analysis.

In [40]:
food_non_null_cols = [food_info.loc[col]['name'] for col in food_info.index if food_info.loc[col]['num_nans'] == 0]

In [41]:
food_non_null = food_data[food_non_null_cols].copy()

In [42]:
food_non_null.columns

Index(['CensusTract', 'State', 'County', 'Urban', 'Pop2010', 'OHU2010',
       'GroupQuartersFlag', 'NUMGQTRS', 'PCTGQTRS', 'LILATracts_1And10',
       'LILATracts_halfAnd10', 'LILATracts_1And20', 'LILATracts_Vehicle',
       'HUNVFlag', 'LowIncomeTracts', 'PovertyRate', 'LA1and10', 'LAhalfand10',
       'LA1and20', 'LATracts_half', 'LATracts1', 'LATracts10', 'LATracts20',
       'LATractsVehicle_20', 'TractLOWI', 'TractKids', 'TractSeniors',
       'TractWhite', 'TractBlack', 'TractAsian', 'TractNHOPI', 'TractAIAN',
       'TractOMultir', 'TractHispanic', 'TractHUNV', 'TractSNAP', 'STATEFP10',
       'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'MTFCC10',
       'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10'],
      dtype='object')

Some of these columns are from the census tract data set and have to do with the locations of the census tracts.  These will not be used here, because we already have information about food access.  The location data will be used in other aspects of the analysis.

In [43]:
food_non_null.drop(columns=['STATEFP10', 'COUNTYFP10', 'TRACTCE10', 'GEOID10', 'NAME10', 'NAMELSAD10', 'MTFCC10',
       'FUNCSTAT10', 'ALAND10', 'AWATER10', 'INTPTLAT10', 'INTPTLON10'], inplace=True)

In [44]:
food_non_null[['State', 'County', 'Urban']].value_counts()

State     County       Urban
Illinois  Cook County  1        1312
                       0           2
dtype: int64

In [45]:
food_non_null[food_non_null['Urban'] == 0]

,CensusTract,State,County,Urban,Pop2010,OHU2010,GroupQuartersFlag,NUMGQTRS,PCTGQTRS,LILATracts_1And10,...,TractSeniors,TractWhite,TractBlack,TractAsian,TractNHOPI,TractAIAN,TractOMultir,TractHispanic,TractHUNV,TractSNAP
1080,17031823801,Illinois,Cook County,0,5716,2254,0,11.0,0.19,0,...,1223.0,5355.0,30.0,196.0,2.0,6.0,127.0,305.0,253.0,21.0
1169,17031828503,Illinois,Cook County,0,4627,1477,0,20.0,0.43,0,...,388.0,1352.0,2835.0,12.0,1.0,14.0,413.0,559.0,79.0,433.0


In [46]:
17031823801 in list(obesity_rate_by_census_df['Tract'])

False

In [47]:
17031828503 in list(obesity_rate_by_census_df['Tract'])

False

Only two census tracts are considered rural, and neither is included in the obesity data.  Of course, all have the same state and county.

In [48]:
food_non_null.drop(columns=['State', 'County', 'Urban'], inplace=True)

### Section D: Merge Health Data with Food Access Data and Export files to csv ###

In [49]:
# The inner merge includes only the census tracts that were included in the community area information.
food_non_null.rename(columns={'CensusTract': 'Tract'}, inplace=True)
food_obesity = pd.merge(left=food_non_null, right=obesity_rate_by_census_df, how='inner', on='Tract')
food_diabetes = pd.merge(left=food_non_null, right=diabetes_rate_by_census_df, how='inner', on='Tract')

In [50]:
food_obesity.to_csv('./data/food_obesity.csv', index=False)
food_diabetes.to_csv('./data/food_diabetes.csv', index = False)